![](https://images.immediate.co.uk/production/volatile/sites/7/2018/01/TIT011DJ_0-345b632.jpg?quality=90&resize=620,413)

In [1]:
import numpy as np
import pandas as pd

In [2]:
trainData = pd.read_csv('../input/train.csv')
testData = pd.read_csv("../input/test.csv")

##### Since RandomForest cannot work with text data , we'll take a look at all the columns, and see which ones are text and which ones** are numeric

In [3]:
print(trainData.dtypes.sort_values())
print(testData.dtypes.sort_values())

PassengerId      int64
Survived         int64
Pclass           int64
SibSp            int64
Parch            int64
Age            float64
Fare           float64
Name            object
Sex             object
Ticket          object
Cabin           object
Embarked        object
dtype: object
PassengerId      int64
Pclass           int64
SibSp            int64
Parch            int64
Age            float64
Fare           float64
Name            object
Sex             object
Ticket          object
Cabin           object
Embarked        object
dtype: object


##### we also have to take care of null values also called NaN values so that R.F algorith can process the data properly****

In [4]:
trainData.isnull().sum()[trainData.isnull().sum()>0]

Age         177
Cabin       687
Embarked      2
dtype: int64

##### the first half "trainData.isnull().sum()" calculates all total occurance of NaN data type in each of the colomns of trainData.the secong half "[trainData.isnull().sum()>0]" acts as like a condition on firt hlaf and only produces sum of NaNs where no of Nan values is greater than 0

In [5]:
testData.isnull().sum()[testData.isnull().sum()>0]

Age       86
Fare       1
Cabin    327
dtype: int64

##### now to work with the dataset we'll either have to drop the colomns with null values or sustitute suitable values in their positions to determine wether a colomn is important enough to not drop or use in R.F algo is called feature importance .Here i'll fill the nan values with mean value of that particular column .

In [6]:
trainData.Age=trainData.Age.fillna(trainData.Age.mean())
testData.Age=testData.Age.fillna(trainData.Age.mean())

trainData.Fare=trainData.Fare.fillna(trainData.Fare.mean())
testData.Fare=testData.Fare.fillna(trainData.Fare.mean())


trainData.Embarked=trainData.Embarked.fillna(trainData.Embarked.mode()[0])
testData.Embarked=testData.Embarked.fillna(trainData.Embarked.mode()[0])

##### Taking a look at our tabel 

In [7]:
trainData.head()
testData.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


##### droppig coloums we wont be using

In [8]:
trainData.drop(['PassengerId','Name','Cabin','Ticket'],axis=1,inplace=True)
testData.drop(['PassengerId','Name','Cabin','Ticket'],axis=1,inplace=True)

In [9]:
testData.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


## Converting categorical variables into numerical

##### a categorical variable is a variable that can take on one of a limited, and usually fixed number of possible values, e.g sex has only male and female , In ML we make use of various techniques like one-hot encoding etc to change catagorical columns into numerical columns . i've used the get_dummies function of pandas to change all the catagorical variables at once . you can read more about catagorical variables and how to process them here , the given link only talks about one-hot encoding but you'll get the gist of the idea.

###### Note ot self: below colomn added after error in fitting Randomforest due to mismatch of newly added colomns from caterorical colomns . Prefered way to do "pd.concat"

In [10]:
combined=pd.concat([trainData, testData], sort=False)
print(combined.dtypes.sort_values())

Pclass        int64
SibSp         int64
Parch         int64
Survived    float64
Age         float64
Fare        float64
Sex          object
Embarked     object
dtype: object


reason for creating a combined tabel : due to error at time of fitting the Model , solution of which was found [Here](https://stackoverflow.com/questions/44026832/valueerror-number-of-features-of-the-model-must-match-the-input)


In [11]:
length = trainData.shape[0]
combined=pd.concat([trainData, testData], sort=False)
combined=pd.get_dummies(combined)
trainData=combined[:length]
testData=combined[length:]

trainData.Survived=trainData.Survived.astype('int')

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


##### spliting the dataframe into dependant variables (y) and independant variabeles (x)

In [12]:
x=trainData.drop("Survived",axis=1)
y=trainData['Survived']
xtest=testData.drop("Survived",axis=1)

## Creating a RandomForest Model

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [14]:
RF = RandomForestClassifier(random_state=1)
results = cross_val_score(RF,x,y,scoring='accuracy',cv=5)
print(results)
np.mean(results)


[0.7877095  0.80446927 0.83146067 0.76404494 0.83050847]


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

0.803638572700704

In [15]:
RF.fit(x, y)


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [16]:
print(RF)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)


##### Storing the result in a csv file

In [17]:
predictions=RF.predict(xtest)
column_name = pd.read_csv('../input/test.csv')
output=pd.DataFrame({'PassengerId':column_name['PassengerId'],'Survived':predictions})
output.to_csv('submission.csv', index=False)